# Fine-tune Google FLAN-T5 Small with LoRA

This notebook logs into Hugging Face, loads the `google/flan-t5-small` model for basic inference, and then fine-tunes it on a local JSON file of question–answer pairs using LoRA via the PEFT library.

In [ ]:
!pip install -U --quiet transformers datasets peft accelerate huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

# Log into Hugging Face (follow the popup in the notebook)
notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 'google/flan-t5-small'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print('Model and tokenizer loaded.')

In [ ]:
input_text = 'What is the capital of France?'
inputs = tokenizer(input_text, return_tensors='pt')
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from datasets import load_dataset

# Assumes `qa_data.json` exists in the same directory and contains a list of {"question": ..., "answer": ...}
dataset = load_dataset('json', data_files='qa_data.json', split='train')
print(dataset[0])

In [ ]:
def preprocess(example):
    input_text = example['question']
    target_text = example['answer']
    model_inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(target_text, truncation=True, padding='max_length', max_length=128)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

processed_dataset = dataset.map(preprocess, remove_columns=dataset.column_names, batched=False)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['q', 'v'],
    lora_dropout=0.1,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM,
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='flan-t5-small-lora',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_steps=10,
    evaluation_strategy='no',
    save_strategy='no',
    fp16=True,
)

collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=collator,
)

trainer.train()

In [ ]:
peft_model.save_pretrained('flan_t5_small_lora_adapter')
print('LoRA adapter saved to flan_t5_small_lora_adapter')

In [ ]:
from peft import PeftModel

base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
lora_model = PeftModel.from_pretrained(base_model, 'flan_t5_small_lora_adapter')

input_text = 'Who wrote the play Hamlet?'
inputs = tokenizer(input_text, return_tensors='pt')
outputs = lora_model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# from huggingface_hub import HfApi
# api = HfApi()
# api.upload_folder(
#     repo_id='username/flan-t5-small-lora-adapter',
#     folder_path='flan_t5_small_lora_adapter',
#     repo_type='model',
#     commit_message='Add LoRA adapter'
# )